In [ ]:
def tokenize_inputs(config, tokenizer, examples):
    max_length = config["max_length"]  # 从配置中获取最大长度

    # hacky backward compatible
    different_eos = tokenizer.eos_token != "</s>"  # 检查分词器的eos token是否与标准的不同
    out = {"labels": [], "input_ids": [], "attention_mask": []}  # 初始化输出字典
    for prompt, response in zip(examples["prompt"], examples["response"]):  # 遍历每个提示和响应
        if different_eos:
            if response.count("</s> \n") > 0:
                response = response.replace("</s> \n", f"{tokenizer.eos_token} \n")  # 替换响应中的结束标记

        prompt_len = len(tokenizer(prompt + "\n", return_tensors="pt")["input_ids"][0])  # 获取提示的长度

        # hack if our prompt is super long
        # 如果提示过长，我们需要截断它以确保包含一些标签
        if prompt_len >= max_length // 2:
            # 如果提示过长，进行截断，但确保不超过1024个标记
            new_len = min(max_length // 2, len(prompt) // 2)
            prompt = prompt[:new_len]
            # 获取新的提示长度
            prompt_len = tokenizer(prompt + "\n", return_tensors="pt", max_length=max_length // 2, truncation=True).input_ids.ne(tokenizer.pad_token_id).sum().item()

        assert prompt_len <= max_length // 2, f"prompt length {prompt_len} exceeds max length {max_length}"  # 确认提示长度不超过最大长度的一半

        input_tokens = tokenizer(prompt + "\n" + response + tokenizer.eos_token,
                                 truncation=True, max_length=max_length, return_tensors="pt")["input_ids"].squeeze()  # 获取输入标记序列

        labels = input_tokens.clone()  # 克隆输入标记序列作为标签
        labels[:prompt_len] = -100  # 将提示部分的标签设为-100，以忽略其损失计算
        if len(labels) < max_length:
            # 将标签填充到最大长度，填充值为-100
            labels = torch.cat([labels, torch.full((max_length - len(labels),), -100)])

        assert (labels == -100).sum() < len(labels), f"Labels are all -100, something wrong. prompt length {prompt_len} exceeds max length {max_length}"  # 确认标签不全是-100
        
        if (labels == -100).sum() == len(labels) - 1:
            print(prompt)
            print(response)
            raise

        padded = tokenizer.pad({"input_ids": input_tokens}, padding="max_length", max_length=max_length, return_tensors="pt")  # 填充输入标记序列
        out["labels"].append(labels)  # 将标签添加到输出字典中
        out["input_ids"].append(padded["input_ids"])  # 将输入标记序列添加到输出字典中
        out["attention_mask"].append(padded["attention_mask"])  # 将注意力掩码添加到输出字典中

    out = {k: torch.stack(v) if isinstance(v, list) else v for k, v in out.items()}  # 将列表中的张量堆叠成一个张量

    return out  # 返回处理后的输出字典



tokenize_inputs函数的作用是将给定的文本输入（包括提示和响应）进行分词和处理，以便将它们转化为模型训练所需的格式。具体而言，它负责：

分词：将文本转换为标记序列（token sequence）。
处理特殊标记：确保使用正确的结束标记（eos token）。
截断：在提示长度过长时进行截断，以保证输入序列的长度不超过模型的最大长度。
填充：将输入序列和标签填充到统一的长度（即最大长度）。
构建标签：为训练构建标签，其中提示部分的标签被设为-100，以忽略其损失计算。

In [ ]:
 for prompt, response in zip(examples["prompt"], examples["response"]):  # 遍历每个提示和响应
        if different_eos:
            if response.count("</s> \n") > 0:
                response = response.replace("</s> \n", f"{tokenizer.eos_token} \n")  # 替换响应中的结束标记

这段代码的作用是在数据预处理阶段确保训练数据中的结束符（end-of-sentence token）与当前使用的tokenizer的结束符（eos token）保持一致。具体分析如下：

1. **检查分词器的EOS token与预期是否不同**:
   - 首先，通过比较`tokenizer.eos_token`与`"</s>"`（这是一个常见的结束符表示），判断当前tokenizer的结束符是否与预期的不同。如果不同，说明后续处理需要调整响应文本中的结束符，以确保与tokenizer的EOS token相匹配。

2. **初始化输出字典**:
   - 创建一个字典`out`，用于收集处理后的数据，包括"labels", "input_ids", 和 "attention_mask"。这些将是模型训练所需的格式。

3. **遍历每个样本的提示（prompt）和响应（response）**:
   - 对于数据集中的每一个样本，提取其提示和响应部分。

4. **替换响应中的结束标记**:
   - 当检测到`different_eos`为真，意味着tokenizer的EOS token与预期不符。此时，如果响应文本中包含`"</s> \n"`这样的序列，代码会将其替换为当前tokenizer的EOS token加上换行符（`f"{tokenizer.eos_token} \n"`）。这一操作保证了在生成的输入序列中，用于标记句子结束的符号与tokenizer实际使用的符号一致，避免了训练和预测阶段可能出现的不匹配问题。

综上，这段代码的核心目的是确保数据集中所有响应文本中的结束符标记与模型训练时将使用的tokenizer的EOS token相匹配，这是数据预处理中的一个重要步骤，有助于维护数据的准确性和模型训练的一致性。

In [ ]:
  prompt_len = len(tokenizer(prompt + "\n", return_tensors="pt")["input_ids"][0])  # 获取提示的长度

这段代码的作用是计算经过tokenizer编码后，一个特定提示（prompt）加上换行符的总长度。具体分析如下：

- **tokenizer(prompt + "\n", return_tensors="pt")**: 
  这部分代码使用tokenizer对输入的`prompt`加上一个换行符`\n`进行编码。Tokenizer是自然语言处理中常用的工具，它将文本拆分成token（词元），这些token可以是单词、子词或者特殊字符等，并将它们转换成模型可以理解的数字ID形式。这里使用`return_tensors="pt"`参数指定返回的类型为PyTorch张量，这通常用于后续的深度学习模型输入。

- **["input_ids"][0]**: 
  编码后的结果是一个字典，其中`"input_ids"`键对应的值是token的ID序列。通过访问`[0]`，我们取出第一个（也是唯一一个，因为这里是单个prompt）样本的input IDs序列。

- **len(...)**: 
  最后，通过计算这个ID序列的长度，得到的是提示字符串（包括结尾的换行符）转换为token后的总数量。这个长度信息在很多场景下非常有用，比如在生成任务中确定自注意力机制中mask的形状，或者在编码-解码模型中确定编码部分的输出长度，以便正确地构造解码器的输入等。

因此，`prompt_len`变量存储的就是tokenizer处理后的prompt长度（以token计数），这对于后续处理如填充、截断以及构建模型输入掩码等都是非常关键的信息。

In [ ]:
assert prompt_len <= max_length // 2, f"prompt length {prompt_len} exceeds max length {max_length}"  # 确认提示长度不超过最大长度的一半

这段代码是一个条件判断语句，使用`assert`关键字来确保程序中的某个条件为真。如果条件不满足，则会抛出一个`AssertionError`异常，并打印提供的错误信息。具体到这个语句，它的作用和意义如下：

- **目的**:
  确保提示（prompt）经过tokenizer编码后的长度（以token计数）不超过预定义的最大长度(`max_length`)的一半。这是个常见的做法，尤其是在基于Transformer架构的模型中，特别是当涉及到序列到序列任务（如文本生成）时。

- **逻辑解释**:
  `prompt_len <= max_length // 2` 是检查条件，意味着提示的长度必须小于或等于最大长度除以2的结果。这里使用`//`操作符来进行整数除法，确保比较的是整数结果。

- **错误处理**:
  如果`prompt_len`超过了`max_length`的一半，程序会中断并抛出错误，错误信息会显示实际的提示长度`prompt_len`和允许的最大长度`max_length`，帮助开发者或用户快速定位问题。

- **应用场景**:
   这种限制常见于需要为模型输入保留空间的情况，比如在文本生成任务中，除了输入的prompt外，还需要为生成的文本预留空间。模型的最大输入长度是有限的，此限制可以防止输入过长导致内存溢出或不符合模型处理要求的情况。

综上所述，这条语句是一种重要的数据验证手段，确保了输入数据符合模型处理的要求，避免了运行时错误和资源过度消耗的风险。

In [ ]:

        padded = tokenizer.pad({"input_ids": input_tokens}, padding="max_length", max_length=max_length, return_tensors="pt")  # 填充输入标记序列
        out["labels"].append(labels)  # 将标签添加到输出字典中
        out["input_ids"].append(padded["input_ids"])  # 将输入标记序列添加到输出字典中
        out["attention_mask"].append(padded["attention_mask"])  # 将注意力掩码添加到输出字典中

这段代码的作用是在准备模型输入数据时，对文本数据进行预处理，具体包括文本编码、序列填充、以及创建注意力掩码，以便于之后用于训练或推理过程。下面是逐行的解释：

1. **文本编码及填充 (`tokenizer.pad`)**:
   - `tokenizer.pad` 函数接收一个字典，其中包含了待处理的序列（在这个例子中是 `"input_ids"`，由之前对原始文本使用tokenizer编码得到）。
   - 参数 `padding="max_length"` 指定了填充方式，意味着所有样本的序列长度都将被调整到 `max_length`。不足的部分会用特定的填充token填充。
   - `max_length` 设定所有序列的目标长度，保证批次内各序列长度一致，这对于大多数深度学习模型是必要的，因为它们要求固定尺寸的输入张量。
   - `return_tensors="pt"` 表示返回的填充结果是PyTorch tensors，这适合直接用于PyTorch模型的输入。

2. **构建输出字典 (`out`)**:
   - `out["labels"].append(labels)`：这行代码把当前样本的标签（`labels`）追加到输出字典的 `"labels"` 列表中。标签通常用于监督学习，如分类或序列生成任务中的目标序列。
   - `out["input_ids"].append(padded["input_ids"])`：将填充后的输入标记序列添加到输出字典的 `"input_ids"` 列表中。这些是文本经过tokenizer编码并填充（或截断）到统一长度后的token ID序列。
   - `out["attention_mask"].append(padded["attention_mask"])`：将生成的注意力掩码追加到输出字典的 `"attention_mask"` 列表中。注意力掩码用来指示哪些部分是实际的输入token，哪些是填充的token，在自注意力机制中使用，以确保模型只关注真实的输入信息，忽略填充部分，从而提高效率和准确性。

通过以上步骤，代码片段为机器学习模型准备了一组标准化的输入数据，包括了经过填充和编码的输入序列、对应的标签序列以及注意力掩码，这些都是模型训练或预测不可或缺的部分。

In [ ]:
  out = {k: torch.stack(v) if isinstance(v, list) else v for k, v in out.items()}  # 将列表中的张量堆叠成一个张量

    return out  # 返回处理后的输出字典


这段代码的作用是对之前构造的输出字典中的数据进行最后的整理，以便于直接用于模型的训练或推理。具体操作如下：

- **遍历字典项**：首先，它遍历输出字典 `out` 的每一项，其中 `k` 是键（key），`v` 是值（value）。
  
- **判断并操作**：对于每个值 `v`：
  - 如果 `v` 是一个列表，并且列表中的元素是PyTorch张量（通过 `isinstance(v, list)` 判断），则使用 `torch.stack(v)` 函数将这个列表中的所有张量沿着一个新的维度堆叠起来。这意味着如果原先每个样本是一个小张量，现在它们会被整合成一个大的张量，其中新增的维度对应于样本数量。这对于批量处理数据至关重要，因为大多数深度学习框架要求输入数据具有批量维度。
  - 如果 `v` 不是一个列表，则保持原样。这可能是因为某些键（如配置参数）对应的值不是需要堆叠的数据，因此直接保留其原值。

- **返回整理后的字典**：经过上述处理后，原本值为列表的项被转换为了单个张量，而其他类型的值则保持不变。最终，整个处理后的字典被返回。

总结来说，这段代码实现了从分散的样本数据到适合模型输入格式的批量数据的转换，是深度学习管道中数据预处理的关键步骤之一，确保了数据能够高效、正确地送入模型进行处理。